In [2]:
from keras.applications import vgg16
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import InputLayer,Flatten,Dense,BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os

ImportError: cannot import name 'Adam' from 'keras.optimizers' (C:\Users\Mask\anaconda3\lib\site-packages\keras\optimizers.py)

In [2]:
datagen = ImageDataGenerator()

In [3]:
conv_base = vgg16.VGG16(include_top=False,input_shape=(32,32,3))

In [4]:
conv_base.trainable = False

In [5]:
base_dir = "/content/drive/MyDrive/DevanagariHandwrittenCharacterDataset"

In [6]:
training_dir = os.path.join(base_dir,"Train")
cv_dir = os.path.join(base_dir,"cv")
testing_dir = os.path.join(base_dir,"Test")

In [7]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0     

In [9]:
def fetch_conv_base_output(train_cv_test_dir,train_cv_test_size):

  conv_base_output = np.zeros(shape=(train_cv_test_size,1,1,512))
  train_cv_test_labels = np.zeros(shape=(train_cv_test_size,46))

  train_cv_test_datagen = datagen.flow_from_directory(directory=train_cv_test_dir,
                                               target_size=(32,32),classes=os.listdir(train_cv_test_dir),batch_size=40)
  
  i = 0
  batch_size = 40

  for input_images_batch,labels_batch in train_cv_test_datagen:

    conv_base_output_batch = conv_base.predict(input_images_batch)
    conv_base_output[i*batch_size:(i+1)*batch_size] = conv_base_output_batch
    train_cv_test_labels[i*batch_size:(i+1)*batch_size] = labels_batch
    i = i + 1

    print("Processed Batch Number",i)

    if i*batch_size >= train_cv_test_size:
      break

  return conv_base_output,train_cv_test_labels 

In [10]:
conv_base_test_output = fetch_conv_base_output(testing_dir,13800)

Found 13800 images belonging to 46 classes.
Processed Batch Number 1
Processed Batch Number 2
Processed Batch Number 3
Processed Batch Number 4
Processed Batch Number 5
Processed Batch Number 6
Processed Batch Number 7
Processed Batch Number 8
Processed Batch Number 9
Processed Batch Number 10
Processed Batch Number 11
Processed Batch Number 12
Processed Batch Number 13
Processed Batch Number 14
Processed Batch Number 15
Processed Batch Number 16
Processed Batch Number 17
Processed Batch Number 18
Processed Batch Number 19
Processed Batch Number 20
Processed Batch Number 21
Processed Batch Number 22
Processed Batch Number 23
Processed Batch Number 24
Processed Batch Number 25
Processed Batch Number 26
Processed Batch Number 27
Processed Batch Number 28
Processed Batch Number 29
Processed Batch Number 30
Processed Batch Number 31
Processed Batch Number 32
Processed Batch Number 33
Processed Batch Number 34
Processed Batch Number 35
Processed Batch Number 36
Processed Batch Number 37
Pro

In [19]:
def build_network():

  network = Sequential()
  network.add(InputLayer(input_shape=(1,1,512)))
  network.add(Flatten())
  network.add(Dense(units=512,activation="relu"))
  network.add(Dense(units=46,activation="softmax"))

  network.compile(optimizer=Adam(),loss="categorical_crossentropy",metrics=["accuracy"])

  return network

In [14]:
conv_base_test_output[1].shape

(13800, 46)

In [15]:
conv_base_test_output_imgs = conv_base_test_output[0]

test_labels = conv_base_test_output[1]

In [29]:
images_tensor_file_handle = open("/content/drive/MyDrive/Colab Notebooks/conv_base_output_imgs.npz",mode="w")

labels_file_handle = open("/content/drive/MyDrive/Colab Notebooks/test_labels.npz",mode="w")

In [1]:
np.savez("/content/drive/MyDrive/Colab Notebooks/conv_base_output_imgs.npz",conv_base_test_output_imgs)

np.savez("/content/drive/MyDrive/Colab Notebooks/test_labels.npz",test_labels)

NameError: name 'np' is not defined

In [17]:
test_datagen = datagen.flow(x=conv_base_test_output_imgs,y=test_labels,batch_size=40)

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/numpy_array_iterator.py:136: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axis 3. However, it was passed an array with shape (13800, 1, 1, 512) (512 channels).
  str(self.x.shape[channels_axis]) + ' channels).')


In [26]:
network = build_network()

In [21]:
network.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_3 (Dense)              (None, 46)                23598     
Total params: 286,254
Trainable params: 286,254
Non-trainable params: 0
_________________________________________________________________


In [27]:
network.fit_generator(generator=test_datagen,epochs=35,workers=2,use_multiprocessing=True)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/35
345/345 [==============================] - 3s 6ms/step - loss: 5.1659 - accuracy: 0.3657
Epoch 2/35
345/345 [==============================] - 2s 6ms/step - loss: 1.0832 - accuracy: 0.7040
Epoch 3/35
345/345 [==============================] - 2s 6ms/step - loss: 0.7551 - accuracy: 0.7708
Epoch 4/35
345/345 [==============================] - 2s 6ms/step - loss: 0.5860 - accuracy: 0.8200
Epoch 5/35
345/345 [==============================] - 2s 6ms/step - loss: 0.5364 - accuracy: 0.8382
Epoch 6/35
345/345 [==============================] - 2s 6ms/step - loss: 0.4373 - accuracy: 0.8592
Epoch 7/35
345/345 [==============================] - 2s 6ms/step - loss: 0.3980 - accuracy: 0.8714
Epoch 8/35
345/345 [==============================] - 2s 6ms/step - loss: 0.3605 - accuracy: 0.8834
Epoch 9/35
345/345 [==============================] - 2s 6ms/step - loss: 0.3331 - accuracy: 0.8884
Epoch 10/35
345/345 [==============================] - 2s 6ms/step - loss: 0.3135 - accuracy: 0.9007